<a href="https://colab.research.google.com/github/yangricardo/gama-accenture-data-warriors-2021/blob/main/DataWarriors_Yang_2021_03_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataWarriors
Armazenamento de dados d COVID-19 de todos os países do mundo através da API: https://documenter.getpostman.com/view/10808728/SzS8rjbc.

1. Crie um Script SQL para criação de um DataBase com um Schema para armazenar os registros de países e os dados de COVID-19 por todo o mundo. Na tabela que será armazenada os dados de países, 2 campos são obrigatórios de serem consistidos:
Nome do país
Código ISO2
Em outros repositórios devem ser armazenados a quantidade de casos confirmados e mortes de cada um dos países do mundo, desde o dia 01/01/2020.
2.  Crie um banco de dados relacional no provedor de nuvem Azure para armazenamento dos dados em questão, estabelecidos pelo script com o dito schema, criado na etapa anterior. O banco de dados pode ser SQL Server, MySQL, MariaDB, Postgres ou algum outro SQL.
3.  Desenvolva um script Python que faça leitura da API determinada no enunciado inicial desta atividade para realizar o armazenamento de países e dos casos confirmados e de mortes da COVID-19. O armazenamento destas informações deverá ser em BD SQL, consistido no Azure através do schema definido na etapa 1 desta atividade.
Após armazenamento dos valores no BD, este dito script Python deverá retornar as seguintes informações em tela, caso o usuário escolha:
  1.  Panorama diário de quantidade de casos confirmados de COVID-19 dos 10 países do mundo com maiores números.
  2. Panorama diário de quantidade de mortes de COVID-19 dos 10 países do mundo com números.
  3. Total de mortes por COVID-19 dos 10 países do mundo com maiores números.
  4.  Total de casos confirmados por COVID-19 dos 10 países do mundo com maiores números.
A impressão das 4 informações citadas acima deverá acontecer em tela, através do prompt de comando de execução do programa.

Parte 2: Em breve

## Instalação de dependências do Postgres

Pode ser necessário apertar o botão `restart runtime` para confirmar a instalação dos arquivos binários das dependências



In [1]:
!pip install psycopg2 psycopg2-binary

     |████████████████████████████████| 3.0MB 5.1MB/s 


## Classe de conexão com Postgres
- O construtor já se conecta por padrão com a instância `data-warriors-postgresql` provisionada em um `Servidor Banco de Dados do Azure para PostgreSQL`

In [133]:
import psycopg2
class Postgres():
  def __init__(self,database='postgres',user='DataWarriorsAdmin@data-warriors-postgresql',password='DataWarriorsPassword!',host='data-warriors-postgresql.postgres.database.azure.com',port='5432'):
    self.connection = psycopg2.connect(host=host, database=database,user=user, password=password,port=port, sslmode='require')
    self.new_cursor()

  def get_connection(self):
    return self.connection

  def new_cursor(self):
    self.cursor = self.connection.cursor()
    return self.cursor

  def commit(self):
    self.connection.commit()

  def rollback(self):
    self.connection.rollback()

  def fetchall(self):
    return self.cursor.fetchall()

  def execute(self,query):
    self.cursor.execute(query)
    self.commit()

  def search(self,query):
    self.new_cursor()
    self.cursor.execute(query)
    self.commit()
    return self.fetchall()

  def insertMany(self,query,data_list):
    try:
      self.new_cursor()
      self.cursor.executemany(query, data_list)
      self.commit()    
      print('Execução da transação conclída com sucesso:', query)
    except Exception as error:
      self.rollback()
      print('Transação falhou:',error)
    
  def closeConnection(self):
    self.connection.close()

## Inicia objeto de conexão com postgres



In [134]:
p = Postgres()

## Executa uma única query de consulta

In [ ]:
p.new_cursor()
p.execute('select * from newtable_1')
p.fetchall()

[('Teste',), ('Teste',)]

## Executa query de insert

In [ ]:
teste="Teste"
p.new_cursor()
p.execute(f"INSERT INTO newtable_1(teste) VALUES ('{teste}')")

## Abstração para execução de query select


In [ ]:
p.search('select * from newtable_1')

[('Teste',), ('Teste',)]

## Abstração para query de inserir vários elementos

In [ ]:
testes = [('teste1',),('teste2',)]
testes_insert_query = """INSERT INTO newtable_1(teste) VALUES(%s)"""
p.insertMany(testes_insert_query,testes)

In [ ]:
p.search('select * from newtable_1')

[('Teste',),
 ('Teste',),
 ('teste1',),
 ('teste2',),
 ('Teste',),
 ('teste1',),
 ('teste2',),
 ('Teste',),
 ('teste1',),
 ('teste2',),
 ('teste1',),
 ('teste2',)]

## Países

In [13]:
import requests
import pandas as pd
import json

In [5]:
countries_json = requests.get('https://api.covid19api.com/countries').json()

In [7]:
countries_df = pd.DataFrame(data=countries_json).drop_duplicates()
countries_df

,Country,Slug,ISO2
0,Australia,australia,AU
1,Netherlands Antilles,netherlands-antilles,AN
2,American Samoa,american-samoa,AS
3,Antigua and Barbuda,antigua-and-barbuda,AG
4,Poland,poland,PL
...,...,...,...
243,Niger,niger,NE
244,Saint-Martin (French part),saint-martin-french-part,MF
245,Slovakia,slovakia,SK
246,Montenegro,montenegro,ME


### Verifica tamanho máximo para coluna Country

In [8]:
 countries_df['Country'].apply(lambda r: len(str(r)) if r!=None else 0).max()

44

## Verifica tamanho máximo para coluna slug

In [9]:
 countries_df['Slug'].apply(lambda r: len(str(r)) if r!=None else 0).max()

44

### Criação da Tabela de Países / Country

In [ ]:
create_table_country_sql = """
CREATE TABLE COUNTRY(
  ISO2      VARCHAR(2)    NOT NULL,
  COUNTRY   VARCHAR(44)   NOT NULL,
  SLUG      VARCHAR(44)   NOT NULL,
  CONSTRAINT PK_COUNTRY PRIMARY KEY (ISO2)
)
"""
p.execute(create_table_country_sql)

In [ ]:
countries_insert_many_data = list(map(lambda country: (country['ISO2'],country['Country'],country['Slug'],), countries_json))
p.insertMany("INSERT INTO COUNTRY(ISO2,COUNTRY,SLUG) VALUES(%s,%s,%s)",countries_insert_many_data)

In [34]:
class ParseDFToDatabase(object):
  def __init__(self,df,table_name):
    self.df = df
    self.table_name = table_name
    self.tuple_list = self.dfToTupleList()
    self.columns = tuple(df.columns.to_list())
    self.format_colums = str(tuple([''.join('%s') for column in self.columns])).replace("'","")
    self.columns = str(tuple(self.columns)).replace("'","").upper()
    self.insert_query = f'INSERT INTO {self.table_name}{self.columns} VALUES{self.format_colums}'
  
  def dfToTupleList(self):
    df_json = json.loads(self.df.to_json(orient='records'))
    return list(map(lambda row: tuple(row.values()),df_json ))

  def get(self):
    return self.insert_query, self.tuple_list

In [130]:
parsed_df = ParseDFToDatabase(countries_df, 'country')
parsed_df.get()[0],parsed_df.get()[1][:5]

('INSERT INTO country(COUNTRY, SLUG, ISO2) VALUES(%s, %s, %s)',
 [('Australia', 'australia', 'AU'),
  ('Netherlands Antilles', 'netherlands-antilles', 'AN'),
  ('American Samoa', 'american-samoa', 'AS'),
  ('Antigua and Barbuda', 'antigua-and-barbuda', 'AG'),
  ('Poland', 'poland', 'PL')])

In [136]:
p.insertMany(parsed_df.get()[0],parsed_df.get()[1])

Execução da transação conclída com sucesso: INSERT INTO country(COUNTRY, SLUG, ISO2) VALUES(%s, %s, %s)
